# Applying Transport Theory to Problems in Computer Graphics 

Given two distributions of material, probability, or whatnot, we might often be interested in computing the distance between the distributions. Our initial idea might be to consider the distributions to be vectors and simply compute the Euclidean distance. However, this does not tell us how we would have to move around "stuff" in order to transform one distribution into the other. 

Computing the Wasserstein distance on the other hand requires us to find a coupling, i.e. a least-work plan for moving material from one distribution such that it turns into the other distribution. 

Unfortunately, the Wasserstein distance, also known as Earth-Mover's Distance, is expensive to compute, but, recently, efficient algorithms to approximate this distance have emerged. 

For a somewhat deeper discussion, please see: 

http://www2.compute.dtu.dk/~janba/w2.html 

The Wasserstein distance and associated coupling have many applications. In computer graphics we could potentially use it to turn a shape into another shape, or an image into another image. It could be used to change color histograms, or compute motion blur or possibly speed up rendering. 

The goal of the project is to implement one or more efficient methods for computing the Wasserstein distance (also known as Earth Movers Distance) and then explore applications such as those outlined above. 


### Andreas' Python implementation.

In [192]:
# Import the packages we will need and setup for rendering
from math import *
import random
import numpy as np
import PIL
from scipy import misc
from scipy.optimize import linprog
from scipy.ndimage.filters import gaussian_filter
from numpy.linalg import norm
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import row, column
from bokeh.io import output_notebook
output_notebook()
def rand_color():
    v = np.array((random.random(), random.random(), random.random()))
    v**2
    v /= np.max(v)
    return v

def color_image(img, col):
    img_out = np.empty(img.shape, dtype=np.uint32)
    view = img_out.view(dtype=np.uint8).reshape(img.shape + (4,))
    for i in range(view.shape[0]):
        for j in range(view.shape[1]):
            alpha = min(1.0, img[i,j] / 0.002)
            view[i, j, 0] = 255.0 * col[0] * alpha
            view[i, j, 1] = 255.0 * col[1] * alpha
            view[i, j, 2] = 255.0 * col[2] * alpha
            view[i, j, 3] = 255* alpha
    return img_out

def color_image_combine(a,b):
    viewa = a.view(dtype=np.uint8).reshape(a.shape + (4,))
    viewb = b.view(dtype=np.uint8).reshape(a.shape + (4,))
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            if viewa[i,j,3] < viewb[i,j,3]:
                viewa[i,j,:] = viewb[i,j,:]



def boxes(shape):
    imgs = []
    w2 = int(shape[0]/2)
    h2 = int(shape[1]/2)
    rngs = [(0,0),(w2,0),(0,h2),(w2,h2)]
    for r in rngs:
        img = np.zeros(shape)
        for i in range(r[0],r[0]+w2):
            for j in range(r[1], r[1]+h2):
                img[i,j] = 1
        imgs.append(img)
    return imgs

def load_image(fn):
    img = misc.imread(fn, True)
    return np.flip(img,0)

sigma = 2.5
def filter(image):
    return gaussian_filter(image, sigma, mode='constant', cval=0.0, truncate=181)

def make_distribution(image_in):
    dist = filter(image_in)
    dist /= np.sum(dist)
    return dist

powv = np.vectorize(pow)
maxv = np.vectorize(max)
logv = np.vectorize(lambda x: log(max(1e-300,x)))

Loading BokehJS ...

In [193]:
# Draw the p, q, and r distributions as vertical bars
pic1 = figure(title="p", plot_width=250, plot_height=250, tools="")
pic2 = figure(title="q", plot_width=250, plot_height=250, tools="")
pic3 = figure(title="r", plot_width=250, plot_height=250, tools="")
p = [0.1,0.5,0.1,0.1,0.1,0.1]
q = [0.1,0.1,0.1,0.5,0.1,0.1]
r = [0.1,0.1,0.1,0.1,0.1,0.5]
pic1.vbar(np.linspace(1,6,6), width=0.9,top=p)
pic2.vbar(np.linspace(1,6,6), width=0.9,top=q)
pic3.vbar(np.linspace(1,6,6), width=0.9,top=r)
show(row(pic1, pic2, pic3))

In [194]:
# Computing the 2-Wasserstein distances from p to p,q, and r
d2 = np.ndarray((6,6))
A_eq = np.zeros((2*6,6*6))
b_eq_pp = np.array(p+p)
b_eq_pq = np.array(p+q)
b_eq_pr = np.array(p+r)
for j in range(0,6):
    for i in range(0,6):
        d2[j,i] = (i-j)**2
        A_eq[j,j*6+i] = 1
        A_eq[6+j,i*6+j] = 1
c = np.reshape(d2,6*6)
output = linprog(c,A_eq=A_eq, b_eq=b_eq_pp)
print("D2(p,p)= %5.2f" % norm(np.array(p)-np.array(p)), " W2(p,p)= %5.2f" % c.dot(output.x))
output = linprog(c,A_eq=A_eq, b_eq=b_eq_pq)
print("D2(p,q)= %5.2f" % norm(np.array(p)-np.array(q)), " W2(p,q)= %5.2f" % c.dot(output.x))
output = linprog(c,A_eq=A_eq, b_eq=b_eq_pr)
print("D2(p,r)= %5.2f" % norm(np.array(p)-np.array(r)), " W2(p,r)= %5.2f" % c.dot(output.x))

D2(p,p)=  0.00  W2(p,p)=  0.00
D2(p,q)=  0.57  W2(p,q)=  1.40
D2(p,r)=  0.57  W2(p,r)=  4.40


In [195]:
import matplotlib.pyplot as plt
# def Sinkhorn(mu):
#     iter = 250
#     v = np.ones(mu[0].shape)
#     w = np.ones(mu[0].shape)
#     for i in range(0,iter):
#         print("-------------------------------")
#         plt.imshow(v)
#         plt.show()
#         print(v.sum())
#         print(v.max())
#         print(v.min())
        
#         v = maxv(filter(w), 1e-300)
#         plt.imshow(v)
#         plt.show()
#         print(v.sum())
#         print(v.max())
#         print(v.min())
        
#         v = mu[0] / v
#         plt.imshow(v)
#         plt.show()
#         print(v.sum())
#         print(v.max())
#         print(v.min())
        
        
#         w = maxv(filter(v), 1e-300)
#         w = mu[1] / w
#     wasserstein_dist = (mu[0]*(logv(v))+mu[1]*(logv(w))).sum()*sigma
#     plt.imshow(w)
#     plt.show()
#     plt.imshow(v)
#     plt.show()
#     print('------------------------')
#     return (wasserstein_dist,v,w)

def Sinkhorn(mu):
    iter = 250
    v = np.ones(mu[0].shape)
    w = np.ones(mu[0].shape)
    for i in range(0,iter):
        v = mu[0] / maxv(filter(w), 1e-300)
        w = mu[1] / maxv(filter(v), 1e-300)
    wasserstein_dist = (mu[0]*(logv(v))+mu[1]*(logv(w))).sum()*sigma
    return (wasserstein_dist,v,w)

In [196]:
# p = np.array(p)
# q = np.array(q)
# r = np.array(r)

# wd_pp, v_pp, w_pp = Sinkhorn([p, p])
# wd_pq, v_pq, w_pq = Sinkhorn([p, q])
# wd_pr, v_pr, w_pr = Sinkhorn([p, r])

# print("W(p,p) = %5.2f" % wd_pp)
# print("W(p,q) = %5.2f" % wd_pq)
# print("W(p,r) = %5.2f" % wd_pr)
# print("------------------------------")
# print(p)
# print(gaussian_filter(p,1.0,mode='reflect',truncate=32))
# print(gaussian_filter(p,1.0,mode='constant',truncate=32))
# print(gaussian_filter(p,1.0,mode='nearest',truncate=32))
# print(gaussian_filter(p,1.0,mode='mirror',truncate=32))
# print(gaussian_filter(p,1.0,mode='wrap',truncate=32))

In [197]:
# Setup code for testing the computation of smooth Wasserstein distances
def move_blob_test():
    dist = [make_distribution(load_image("images/one-blob.png")),
    make_distribution(load_image("images/one-blob-moved.png")),
    make_distribution(load_image("images/one-blob-moved-even-more.png")),
    make_distribution(load_image("images/one-blob-moved-even-more-again.png"))
    ]
    pics = [figure(title = "Original", x_range=(0,1),y_range=(0,1),plot_width=250,plot_height=250)]
    pics[0].image(image=[dist[0]],x=0,y=0,dw=1,dh=1)
    for i in range(1,4):
        wd,v,w = Sinkhorn((dist[0],dist[i]))
        ed = norm(dist[0]-dist[i])
        p = figure(title = "W2 = "+str(wd)+" D2 = "+str(ed), x_range=(0,1),y_range=(0,1),plot_width=250,plot_height=250)
        p.image(image=[dist[i]],x=0,y=0,dw=1,dh=1)
        pics += [p,]
    show(row(pics))

In [198]:
# Compute W2 between several similar distributions
move_blob_test()

/home/jessica/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [162]:
# Code for the evaluation of the coupling matrix and quadrant test

def SinkhornEvalR(v,w,a):
    return v*filter(w*a)

def SinkhornEvalL(v,w,a):
    return w*filter(v*a)

dist1 = make_distribution(load_image("images/smile_s.png"))
dist2 = make_distribution(load_image("images/dots_s.png"))
#output_file("sinkhorn.html")

def quadrant_test():
    # Compute the coupling between two distributions and use that to find
    # out how we go from quadrants in one image to quadrants in the other.
    wd,v,w = Sinkhorn((dist1,dist2))
    print ("Wasserstein distance = ", wd)
    img_out = np.zeros(dist1.shape,dtype=np.uint32)
    cmp_out = np.zeros(dist1.shape,dtype=np.uint32)
    s = 0
    for b in boxes(dist1.shape):
        color = rand_color()
        img = SinkhornEvalR(v,w,b)
        s += img.sum()
        col_img = color_image(img, color)
        cmp_img = color_image(b*dist2, color)
        color_image_combine(img_out, col_img)
        color_image_combine(cmp_out, cmp_img)
        p1 = figure(title = "Original", x_range=(0,1),y_range=(1,0),plot_width=420,plot_height=420)
        p1.image_rgba(image=[cmp_out],x=0,y=1,dw=1,dh=1)
        p2 = figure(title = "Wasserstein distance = "+str(wd), x_range=(0,1),y_range=(1,0),plot_width=420,plot_height=420)
        p2.image_rgba(image=[img_out],x=0,y=1,dw=1,dh=1)
    show(row(p1,p2))
    print(s)

/home/jessica/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [163]:
# Show how four blobs map to a smiley using the coupling from the smoothed W2
quadrant_test()

Wasserstein distance =  54.77155979302002


0.9999999709941912


In [96]:
# Show  smoothed Wasserstein interpolation
def WassersteinBarycenter(alpha, mu, dims):
    v = [np.ones(dims), np.ones(dims)]
    w = [np.ones(dims), np.ones(dims)]
    d = [np.ones(dims), np.ones(dims)]
    mu_out = np.ones(dims)
    iter = 100
    for j in range(0,iter):
        mu_out = np.ones(dims)
        for i in range(0,2):
            w[i] = mu[i] / maxv(filter(v[i]),1e-100)
            d[i] = v[i] * filter(w[i])
            mu_out = mu_out * powv(d[i],alpha[i])
        for i in range(0,2):
            v[i] = v[i] * mu_out/maxv(d[i],1e-100)
    return mu_out



def WassersteinBarycenter_test():
    # Compute a sequence of Wasserstein barycenters interpolating between our two
    # distributions
    dist1 = make_distribution(load_image("images/smile_s.png"))
    dist2 = make_distribution(load_image("images/dots_s.png"))
    rows = []
    pics = []
    for alpha in np.linspace(0,1,8):
        dist = WassersteinBarycenter((alpha,1.0-alpha), (dist1,dist2), dist1.shape)
        p = figure(x_range=(0,1),y_range=(1,0),plot_width=200,plot_height=200)
        p.image([dist],x=0,y=1,dw=1,dh=1)
        pics += [p]
        if len(pics) == 4:
            rows += [row(pics)]
            pics = []
    c = column(rows)
    show(c)

In [32]:
# Interpolate between two distributions
WassersteinBarycenter_test()

/home/jessica/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/jessica/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in true_divide
  from ipykernel import kernelapp as app
/home/jessica/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in multiply
  if sys.path[0] == '':
